In [ ]:
import pandas as pd

df_2005sucio = pd.read_csv('*.csv', delimiter=';', encoding='latin1')
df_2006 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2007 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2008 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2009 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2010 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2011 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2012 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2013 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2014 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2015 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2016 = pd.read_csv('*', delimiter=';', encoding='latin1')
df_2017 = pd.read_csv('*', delimiter=';')
df_2018 = pd.read_csv('*', delimiter=';')
df_2019 = pd.read_csv('*', delimiter=';')
df_2020sucio = pd.read_csv(r'*', encoding='ISO-8859-1', delimiter=';')



In [2]:
df_2005 = df_2005sucio.dropna()
df_2020sucio2 = df_2020sucio.rename(columns={
    'CPF_BENEFICIARIO': 'CPF_BENEFICIARIO_BOLSA',
    'RACA_BENEFICIARIO': 'RACA_BENEFICIARIO_BOLSA',
    'SEXO_BENEFICIARIO': 'SEXO_BENEFICIARIO_BOLSA',
    'DATA_NASCIMENTO': 'DT_NASCIMENTO_BENEFICIARIO',
    'REGIAO_BENEFICIARIO': 'REGIAO_BENEFICIARIO_BOLSA',
    'MUNICIPIO_BENEFICIARIO': 'MUNICIPIO_BENEFICIARIO_BOLSA',
    'UF_BENEFICIARIO': 'SIGLA_UF_BENEFICIARIO_BOLSA',
    })
df_2020 = df_2020sucio2.drop(columns=['MUNICIPIO', 'CAMPUS'])



In [3]:
# Concatenar os DataFrames de 2005 a 2020
df_total_sucio = pd.concat([
    df_2005, df_2006, df_2007, df_2008, df_2009,
    df_2010, df_2011, df_2012, df_2013, df_2014,
    df_2015, df_2016, df_2017, df_2018, df_2019, df_2020
], ignore_index=True)

# Remover linhas com valores ausentes nas colunas essenciais e criar uma cópia segura
df_totalsucio2 = df_total_sucio.dropna(subset=[
    'ANO_CONCESSAO_BOLSA', 'CODIGO_EMEC_IES_BOLSA'
]).copy()

# Substituir valores na coluna de sexo
df_totalsucio2.loc[:, 'SEXO_BENEFICIARIO_BOLSA'] = df_totalsucio2['SEXO_BENEFICIARIO_BOLSA'].replace({
    'Feminino': 'F', 
    'Masculino': 'M'
})

# Substituir valores na coluna de deficiência
df_totalsucio2.loc[:, 'BENEFICIARIO_DEFICIENTE_FISICO'] = df_totalsucio2['BENEFICIARIO_DEFICIENTE_FISICO'].replace({
    'SIM': 'S', 
    'NÃO': 'N'
})
df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'] = df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'].str.replace('-', '/')
# Converter a coluna para datetime, se ainda não for
df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'] = pd.to_datetime(df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'], errors='coerce')
# Formatando para o estilo desejado, por exemplo: 'dd/mm/yyyy'
df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'] = df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'].dt.strftime('%d/%m/%Y')
df_total = df_totalsucio2.dropna()



C:\Users\Lucan Vieira\AppData\Local\Temp\ipykernel_5376\97993319.py:26: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'] = pd.to_datetime(df_totalsucio2['DT_NASCIMENTO_BENEFICIARIO'], errors='coerce')


In [4]:
# Para a tabela de localidades
dim_localidade = df_total[['REGIAO_BENEFICIARIO_BOLSA', 'SIGLA_UF_BENEFICIARIO_BOLSA', 'MUNICIPIO_BENEFICIARIO_BOLSA']].dropna().drop_duplicates()

# Para a tabela de instituições
dim_instituicao = df_total[['CODIGO_EMEC_IES_BOLSA', 'NOME_IES_BOLSA']].dropna().drop_duplicates()

# Para a tabela de cursos
dim_curso = df_total[['NOME_CURSO_BOLSA', 'MODALIDADE_ENSINO_BOLSA', 'NOME_TURNO_CURSO_BOLSA']].dropna().drop_duplicates()

# Para a tabela de beneficiários
dim_beneficiario = df_total[['CPF_BENEFICIARIO_BOLSA', 'SEXO_BENEFICIARIO_BOLSA', 
                             'RACA_BENEFICIARIO_BOLSA', 'DT_NASCIMENTO_BENEFICIARIO', 
                             'BENEFICIARIO_DEFICIENTE_FISICO']].dropna().drop_duplicates()



In [5]:

# Criar IDs únicos
dim_instituicao['instituicao_id'] = (dim_instituicao.reset_index().index + 1).astype(int)
dim_curso['curso_id'] = (dim_curso.reset_index().index + 1).astype(int)
dim_beneficiario['beneficiario_id'] = (dim_beneficiario.reset_index().index + 1).astype(int)
dim_localidade['localidade_id'] = (dim_localidade.reset_index().index + 1).astype(int)


In [6]:
import hashlib

# Merge com o DataFrame total para construir a tabela fato
df_fato = df_total \
    .merge(dim_instituicao, on=['CODIGO_EMEC_IES_BOLSA', 'NOME_IES_BOLSA'], how='left') \
    .merge(dim_curso, on=['NOME_CURSO_BOLSA', 'MODALIDADE_ENSINO_BOLSA', 'NOME_TURNO_CURSO_BOLSA'], how='left') \
    .merge(dim_beneficiario, on=[ 
        'CPF_BENEFICIARIO_BOLSA', 'SEXO_BENEFICIARIO_BOLSA', 
        'RACA_BENEFICIARIO_BOLSA', 'DT_NASCIMENTO_BENEFICIARIO', 
        'BENEFICIARIO_DEFICIENTE_FISICO'], how='left') \
    .merge(dim_localidade, on=['REGIAO_BENEFICIARIO_BOLSA', 'SIGLA_UF_BENEFICIARIO_BOLSA', 'MUNICIPIO_BENEFICIARIO_BOLSA'], how='left')

# Após o merge, substitua NaN por um valor padrão (como 0 ou outro valor adequado)
# Alterar as colunas para o tipo Int64, que permite valores nulos (NaN)
df_fato['instituicao_id'] = df_fato['instituicao_id'].astype('Int64')
df_fato['curso_id'] = df_fato['curso_id'].astype('Int64')
df_fato['beneficiario_id'] = df_fato['beneficiario_id'].astype('Int64')
df_fato['localidade_id'] = df_fato['localidade_id'].astype('Int64')

# Verificar valores nulos na coluna 'instituicao_id'
df_fato[df_fato['instituicao_id'].isna()]

# Se o ANO_CONCESSAO_BOLSA tiver valores float, converta para inteiro também
df_fato['ANO_CONCESSAO_BOLSA'] = df_fato['ANO_CONCESSAO_BOLSA'].fillna(0).astype(int)


# Criar a tabela fato a partir das colunas selecionadas
fato_bolsa = df_fato[['ANO_CONCESSAO_BOLSA', 'instituicao_id', 'curso_id', 'beneficiario_id', 'localidade_id', 'TIPO_BOLSA']]
fato_bolsa = fato_bolsa.reset_index(drop=True)

# Função para gerar um hash único baseado nos campos
def gerar_chave_composta(row):
    chave_str = f"{row['ANO_CONCESSAO_BOLSA']}_{row['instituicao_id']}_{row['curso_id']}_{row['beneficiario_id']}_{row['localidade_id']}_{row['TIPO_BOLSA']}"
    return hashlib.sha256(chave_str.encode('utf-8')).hexdigest()

# Aplicar a função e criar a coluna fato_id
fato_bolsa['fato_id'] = fato_bolsa.apply(gerar_chave_composta, axis=1)



In [7]:
# Substituindo 'Indigena' por 'Indígena' na coluna 'RACA_BENEFICIARIO_BOLSA'
dim_beneficiario['RACA_BENEFICIARIO_BOLSA'] = dim_beneficiario['RACA_BENEFICIARIO_BOLSA'].replace('Ind¡gena', 'Indígena')


In [8]:
dim_beneficiario

,CPF_BENEFICIARIO_BOLSA,SEXO_BENEFICIARIO_BOLSA,RACA_BENEFICIARIO_BOLSA,DT_NASCIMENTO_BENEFICIARIO,BENEFICIARIO_DEFICIENTE_FISICO,beneficiario_id
0,***264740**,F,Branca,17/02/1987,N,1
1,***976940**,F,Parda,14/06/1986,N,2
2,***045340**,F,Parda,06/03/1984,N,3
3,***988420**,M,Branca,19/10/1987,N,4
4,***378080**,M,Amarela,20/07/1987,N,5
...,...,...,...,...,...,...
2873119,062.XXX.XXX-70,F,Parda,30/11/1996,N,2533675
2873120,177.XXX.XXX-80,F,Parda,25/03/2002,N,2533676
2873121,042.XXX.XXX-54,M,Branca,19/04/1997,N,2533677
2873122,601.XXX.XXX-53,F,Parda,17/09/1975,N,2533678
